# Image Processing Steps - License Plate Detection

Notebook này trình bày các bước xử lý ảnh để phát hiện biển số xe.

## 🚀 Hướng dẫn sử dụng trên Google Colab:

1. Chạy các cell theo thứ tự từ trên xuống
2. Ở cell "Upload ảnh", chọn file ảnh biển số để upload
3. Các bước xử lý sẽ tự động hiển thị kết quả

## 📋 Các bước xử lý:

1. **Resize ảnh** về kích thước chuẩn (1920x1080)
2. **Chuyển đổi sang Grayscale** (HSV Value channel)
3. **Tăng độ tương phản** (Top Hat + Black Hat morphology)
4. **Làm mịn ảnh** (Gaussian Blur)
5. **Nhị phân hóa** (Adaptive Threshold)
6. **Phát hiện cạnh** (Canny Edge Detection)
7. **Dilation** (nối các cạnh bị đứt đoạn)
8. **Tìm contour** và phát hiện biển số

**Lưu ý:** Notebook này chỉ xử lý ảnh và phát hiện biển số, không bao gồm phần nhận dạng ký tự.


## 1. Kiểm tra môi trường


In [ ]:
# Tự động phát hiện môi trường Colab
try:
    import google.colab
    IN_COLAB = True
    print("🌐 Đang chạy trên Google Colab")
except ImportError:
    IN_COLAB = False
    print("💻 Đang chạy trên máy local (Jupyter)")


## 2. Import Libraries và định nghĩa các class xử lý ảnh


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math

plt.rcParams['figure.figsize'] = (15, 10)
plt.rcParams['image.cmap'] = 'gray'

class ImagePreprocessor:
    GAUSSIAN_SMOOTH_FILTER_SIZE = (5, 5)
    ADAPTIVE_THRESH_BLOCK_SIZE = 19
    ADAPTIVE_THRESH_WEIGHT = 9
    MORPHOLOGY_ITERATIONS = 10
    MORPHOLOGY_KERNEL_SIZE = (3, 3)
    
    def extract_value(self, img_original):
        img_hsv = cv2.cvtColor(img_original, cv2.COLOR_BGR2HSV)
        _, _, img_value = cv2.split(img_hsv)
        return img_value
    
    def maximize_contrast(self, img_grayscale):
        structuring_element = cv2.getStructuringElement(
            cv2.MORPH_RECT, 
            self.MORPHOLOGY_KERNEL_SIZE
        )
        
        img_top_hat = cv2.morphologyEx(
            img_grayscale, 
            cv2.MORPH_TOPHAT, 
            structuring_element, 
            iterations=self.MORPHOLOGY_ITERATIONS
        )
        
        img_black_hat = cv2.morphologyEx(
            img_grayscale, 
            cv2.MORPH_BLACKHAT, 
            structuring_element, 
            iterations=self.MORPHOLOGY_ITERATIONS
        )
        
        img_plus_top_hat = cv2.add(img_grayscale, img_top_hat)
        img_enhanced = cv2.subtract(img_plus_top_hat, img_black_hat)
        
        return img_enhanced


class PlateDetector:
    CANNY_THRESHOLD_LOW = 250
    CANNY_THRESHOLD_HIGH = 255
    DILATION_KERNEL_SIZE = (3, 3)
    DILATION_ITERATIONS = 1
    APPROX_POLY_EPSILON_FACTOR = 0.06
    MAX_CONTOURS = 10
    TARGET_SIZE = (1920, 1080)
    
    def resize_image(self, img):
        return cv2.resize(img, self.TARGET_SIZE)
    
    def detect_edges(self, img_thresh):
        return cv2.Canny(
            img_thresh,
            self.CANNY_THRESHOLD_LOW,
            self.CANNY_THRESHOLD_HIGH
        )
    
    def dilate_edges(self, canny_image):
        kernel = np.ones(self.DILATION_KERNEL_SIZE, np.uint8)
        return cv2.dilate(canny_image, kernel, iterations=self.DILATION_ITERATIONS)
    
    def find_plate_contours(self, dilated_image):
        contours, _ = cv2.findContours(
            dilated_image,
            cv2.RETR_TREE,
            cv2.CHAIN_APPROX_SIMPLE
        )
        
        contours = sorted(contours, key=cv2.contourArea, reverse=True)[:self.MAX_CONTOURS]
        
        plate_contours = []
        for contour in contours:
            peri = cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(
                contour,
                self.APPROX_POLY_EPSILON_FACTOR * peri,
                True
            )
            
            if len(approx) == 4:
                plate_contours.append(approx)
        
        return plate_contours

preprocessor = ImagePreprocessor()
detector = PlateDetector()

print("✅ Modules initialized successfully!")
print(f"📐 Target size: {detector.TARGET_SIZE}")
print(f"🔲 Gaussian kernel size: {preprocessor.GAUSSIAN_SMOOTH_FILTER_SIZE}")
print(f"📊 Adaptive threshold block size: {preprocessor.ADAPTIVE_THRESH_BLOCK_SIZE}")


## 3. Load Image


### Upload ảnh (Google Colab)


In [ ]:
from google.colab import files
from io import BytesIO
from PIL import Image

print("📤 Vui lòng chọn file ảnh để upload...")
uploaded = files.upload()

image_filename = list(uploaded.keys())[0]
image_bytes = uploaded[image_filename]
image_pil = Image.open(BytesIO(image_bytes))
img_original = cv2.cvtColor(np.array(image_pil), cv2.COLOR_RGB2BGR)

print(f"✅ Đã upload ảnh: {image_filename}")
print(f"📐 Kích thước: {img_original.shape}")


### Load ảnh từ file (Tùy chọn)


In [ ]:
if 'img_original' not in locals() or img_original is None:
    possible_paths = [
        "/content/data/test_images/1.jpg",
        "/content/Image-Processing/data/test_images/1.jpg",
        "1.jpg",
        "test_image.jpg"
    ]
    
    img_original = None
    for path in possible_paths:
        img_original = cv2.imread(path)
        if img_original is not None:
            print(f"✅ Đã load ảnh từ: {path}")
            break
    
    if img_original is None:
        print("⚠️ Không tìm thấy ảnh. Vui lòng upload ảnh ở cell trước.")
        raise FileNotFoundError("Không tìm thấy ảnh để xử lý.")

img_original_rgb = cv2.cvtColor(img_original, cv2.COLOR_BGR2RGB)

print(f"📐 Kích thước: {img_original.shape}")
print(f"📊 Dtype: {img_original.dtype}")

plt.figure(figsize=(12, 8))
plt.imshow(img_original_rgb)
plt.title("Ảnh gốc (Original Image)", fontsize=16, fontweight='bold')
plt.axis('off')
plt.tight_layout()
plt.show()


## 4. Bước 1: Resize ảnh về kích thước chuẩn


In [ ]:
img_resized = detector.resize_image(img_original)
img_resized_rgb = cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB)

print(f"📐 Kích thước sau resize: {img_resized.shape}")

plt.figure(figsize=(12, 8))
plt.imshow(img_resized_rgb)
plt.title(f"Bước 1: Resize về kích thước {detector.TARGET_SIZE}", fontsize=16, fontweight='bold')
plt.axis('off')
plt.tight_layout()
plt.show()


## 5. Bước 2: Chuyển đổi sang Grayscale (HSV Value)


In [ ]:
img_grayscale = preprocessor.extract_value(img_resized)

print(f"📐 Kích thước: {img_grayscale.shape}")
print(f"📊 Min: {img_grayscale.min()}, Max: {img_grayscale.max()}")

plt.figure(figsize=(12, 8))
plt.imshow(img_grayscale, cmap='gray')
plt.title("Bước 2: Chuyển đổi sang Grayscale (HSV Value channel)", fontsize=16, fontweight='bold')
plt.axis('off')
plt.tight_layout()
plt.show()


## 6. Bước 3: Tăng độ tương phản (Top Hat + Black Hat)


In [ ]:
img_contrast = preprocessor.maximize_contrast(img_grayscale)

print(f"📐 Kích thước: {img_contrast.shape}")
print(f"📊 Min: {img_contrast.min()}, Max: {img_contrast.max()}")

fig, axes = plt.subplots(1, 2, figsize=(16, 8))

axes[0].imshow(img_grayscale, cmap='gray')
axes[0].set_title("Trước khi tăng tương phản", fontsize=14, fontweight='bold')
axes[0].axis('off')

axes[1].imshow(img_contrast, cmap='gray')
axes[1].set_title("Bước 3: Sau khi tăng tương phản (Top Hat + Black Hat)", fontsize=14, fontweight='bold')
axes[1].axis('off')

plt.tight_layout()
plt.show()


## 7. Bước 4: Làm mịn ảnh (Gaussian Blur)


In [ ]:
img_blurred = cv2.GaussianBlur(
    img_contrast, 
    preprocessor.GAUSSIAN_SMOOTH_FILTER_SIZE, 
    0
)

print(f"📐 Kích thước: {img_blurred.shape}")
print(f"🔲 Kernel size: {preprocessor.GAUSSIAN_SMOOTH_FILTER_SIZE}")

fig, axes = plt.subplots(1, 2, figsize=(16, 8))

axes[0].imshow(img_contrast, cmap='gray')
axes[0].set_title("Trước khi blur", fontsize=14, fontweight='bold')
axes[0].axis('off')

axes[1].imshow(img_blurred, cmap='gray')
axes[1].set_title(f"Bước 4: Sau khi blur (Gaussian {preprocessor.GAUSSIAN_SMOOTH_FILTER_SIZE})", fontsize=14, fontweight='bold')
axes[1].axis('off')

plt.tight_layout()
plt.show()


## 8. Bước 5: Nhị phân hóa (Adaptive Threshold)


In [ ]:
img_thresh = cv2.adaptiveThreshold(
    img_blurred,
    255.0,
    cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
    cv2.THRESH_BINARY_INV,
    preprocessor.ADAPTIVE_THRESH_BLOCK_SIZE,
    preprocessor.ADAPTIVE_THRESH_WEIGHT
)

print(f"📐 Kích thước: {img_thresh.shape}")
print(f"🔲 Block size: {preprocessor.ADAPTIVE_THRESH_BLOCK_SIZE}")
print(f"⚖️ Weight: {preprocessor.ADAPTIVE_THRESH_WEIGHT}")

fig, axes = plt.subplots(1, 2, figsize=(16, 8))

axes[0].imshow(img_blurred, cmap='gray')
axes[0].set_title("Trước khi nhị phân hóa", fontsize=14, fontweight='bold')
axes[0].axis('off')

axes[1].imshow(img_thresh, cmap='gray')
axes[1].set_title(f"Bước 5: Nhị phân hóa (Adaptive Threshold, block={preprocessor.ADAPTIVE_THRESH_BLOCK_SIZE})", fontsize=14, fontweight='bold')
axes[1].axis('off')

plt.tight_layout()
plt.show()


## 9. Bước 6: Phát hiện cạnh (Canny Edge Detection)


In [ ]:
img_canny = detector.detect_edges(img_thresh)

print(f"📐 Kích thước: {img_canny.shape}")
print(f"🔍 Canny thresholds: Low={detector.CANNY_THRESHOLD_LOW}, High={detector.CANNY_THRESHOLD_HIGH}")

fig, axes = plt.subplots(1, 2, figsize=(16, 8))

axes[0].imshow(img_thresh, cmap='gray')
axes[0].set_title("Ảnh nhị phân", fontsize=14, fontweight='bold')
axes[0].axis('off')

axes[1].imshow(img_canny, cmap='gray')
axes[1].set_title(f"Bước 6: Phát hiện cạnh (Canny Edge, {detector.CANNY_THRESHOLD_LOW}-{detector.CANNY_THRESHOLD_HIGH})", fontsize=14, fontweight='bold')
axes[1].axis('off')

plt.tight_layout()
plt.show()


## 10. Bước 7: Dilation (Nối các cạnh bị đứt đoạn)


In [ ]:
img_dilated = detector.dilate_edges(img_canny)

print(f"📐 Kích thước: {img_dilated.shape}")
print(f"🔲 Kernel size: {detector.DILATION_KERNEL_SIZE}")
print(f"🔄 Iterations: {detector.DILATION_ITERATIONS}")

fig, axes = plt.subplots(1, 2, figsize=(16, 8))

axes[0].imshow(img_canny, cmap='gray')
axes[0].set_title("Trước khi dilation", fontsize=14, fontweight='bold')
axes[0].axis('off')

axes[1].imshow(img_dilated, cmap='gray')
axes[1].set_title(f"Bước 7: Sau khi dilation (kernel={detector.DILATION_KERNEL_SIZE}, iter={detector.DILATION_ITERATIONS})", fontsize=14, fontweight='bold')
axes[1].axis('off')

plt.tight_layout()
plt.show()


## 11. Bước 8: Tìm contour và phát hiện biển số


In [ ]:
plate_contours = detector.find_plate_contours(img_dilated)

print(f"🔍 Số lượng contour phát hiện được: {len(plate_contours)}")

img_with_contours = img_resized.copy()
for contour in plate_contours:
    cv2.drawContours(img_with_contours, [contour], -1, (0, 255, 0), 3)

img_with_contours_rgb = cv2.cvtColor(img_with_contours, cv2.COLOR_BGR2RGB)

fig, axes = plt.subplots(1, 2, figsize=(16, 8))

axes[0].imshow(img_resized_rgb)
axes[0].set_title("Ảnh gốc", fontsize=14, fontweight='bold')
axes[0].axis('off')

axes[1].imshow(img_with_contours_rgb)
axes[1].set_title(f"Bước 8: Phát hiện biển số ({len(plate_contours)} biển số)", fontsize=14, fontweight='bold')
axes[1].axis('off')

plt.tight_layout()
plt.show()

if len(plate_contours) > 0:
    print("\n📋 Chi tiết các contour phát hiện được:")
    for i, contour in enumerate(plate_contours):
        area = cv2.contourArea(contour)
        x, y, w, h = cv2.boundingRect(contour)
        print(f"  Contour {i+1}: Area={area:.0f}, Bounding box=({x}, {y}, {w}, {h})")


## 12. Tổng kết: Hiển thị tất cả các bước


In [ ]:
fig, axes = plt.subplots(2, 4, figsize=(20, 10))
axes = axes.ravel()

axes[0].imshow(img_resized_rgb)
axes[0].set_title("1. Ảnh gốc (Resized)", fontsize=12, fontweight='bold')
axes[0].axis('off')

axes[1].imshow(img_grayscale, cmap='gray')
axes[1].set_title("2. Grayscale (HSV Value)", fontsize=12, fontweight='bold')
axes[1].axis('off')

axes[2].imshow(img_contrast, cmap='gray')
axes[2].set_title("3. Tăng tương phản", fontsize=12, fontweight='bold')
axes[2].axis('off')

axes[3].imshow(img_blurred, cmap='gray')
axes[3].set_title("4. Gaussian Blur", fontsize=12, fontweight='bold')
axes[3].axis('off')

axes[4].imshow(img_thresh, cmap='gray')
axes[4].set_title("5. Adaptive Threshold", fontsize=12, fontweight='bold')
axes[4].axis('off')

axes[5].imshow(img_canny, cmap='gray')
axes[5].set_title("6. Canny Edge", fontsize=12, fontweight='bold')
axes[5].axis('off')

axes[6].imshow(img_dilated, cmap='gray')
axes[6].set_title("7. Dilation", fontsize=12, fontweight='bold')
axes[6].axis('off')

axes[7].imshow(img_with_contours_rgb)
axes[7].set_title(f"8. Phát hiện biển số ({len(plate_contours)})", fontsize=12, fontweight='bold')
axes[7].axis('off')

plt.suptitle("Tổng hợp các bước xử lý ảnh", fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()


## 13. Thông tin các tham số sử dụng


In [ ]:
print("=" * 60)
print("THÔNG TIN CÁC THAM SỐ XỬ LÝ ẢNH")
print("=" * 60)
print(f"\n1. Kích thước ảnh chuẩn: {detector.TARGET_SIZE}")
print(f"\n2. Preprocessing:")
print(f"   - Gaussian kernel size: {preprocessor.GAUSSIAN_SMOOTH_FILTER_SIZE}")
print(f"   - Adaptive threshold block size: {preprocessor.ADAPTIVE_THRESH_BLOCK_SIZE}")
print(f"   - Adaptive threshold weight: {preprocessor.ADAPTIVE_THRESH_WEIGHT}")
print(f"   - Morphology iterations: {preprocessor.MORPHOLOGY_ITERATIONS}")
print(f"   - Morphology kernel size: {preprocessor.MORPHOLOGY_KERNEL_SIZE}")
print(f"\n3. Edge Detection:")
print(f"   - Canny threshold low: {detector.CANNY_THRESHOLD_LOW}")
print(f"   - Canny threshold high: {detector.CANNY_THRESHOLD_HIGH}")
print(f"   - Dilation kernel size: {detector.DILATION_KERNEL_SIZE}")
print(f"   - Dilation iterations: {detector.DILATION_ITERATIONS}")
print(f"   - Max contours: {detector.MAX_CONTOURS}")
print(f"   - Approx poly epsilon factor: {detector.APPROX_POLY_EPSILON_FACTOR}")
print("=" * 60)
